In [11]:
import matplotlib.pyplot as plt
import pandas as pd
from pyts.image import GramianAngularField
import numpy as np
from os import path
from tqdm import tqdm

Filepaths

In [12]:
dataset_dir = path.join('..', '..', 'Data', 'Normalized')
hparams_dir = path.join('..', 'hyperparameters')

## Reading and transforming time series

In [13]:
sensor_types = ['body_acc', 'body_gyro', 'total_acc']
axis = ['x', 'y', 'z']

In [14]:
def get_dataset(col_type, dataset_type, start=0, stop=0):
    if col_type == 'X':
        horizontal_images = list()
        for sensor in sensor_types:
            dataset = list()
            for direction in axis:
                filepath = path.join(dataset_dir, dataset_type, '_'.join([sensor,direction,dataset_type])+'.csv')
                gaf = GramianAngularField(sample_range=None)
                data = pd.read_csv(filepath, delim_whitespace=True, header=None)
                gaf.fit(data)
                dataset.append(gaf.transform(data[start:stop]))
            horizontal_images.append(np.dstack(dataset))
        vertical_images = np.hstack(horizontal_images)
        return vertical_images
    else:
        return pd.read_csv(path.join(dataset_dir, dataset_type, 'y_' + dataset_type + '.txt'), header=None)

In [15]:
classes = {1:'WALKING', 2:'WALKING_UPSTAIRS', 3:'WALKING_DOWNSTAIRS', 4:'SITTING', 5:'STANDING', 6:'LAYING'}

In [16]:
y_train = get_dataset('y', 'train')

In [17]:
y_test = get_dataset('y', 'test')

## Saving Images

In [23]:
def save_images_by_class(dataset_type, y):
    prev = 0
    count = 1
    if dataset_type == 'train':
        start = 919
        stop = 7353
        step = 919
    else:
        start = 1474
        stop = 2949
        step = 1474
    for i in range(start, stop, step):
        X = get_dataset('X', dataset_type, prev, i)
        for j in tqdm(range(len(X))):
            plt.imsave(path.join('..', '..', 'Data', 'normalizedImages', dataset_type, classes[y[0][count-1]], str(count)+'.png'), X[j], cmap="jet")
            count += 1
        prev = i

In [24]:
save_images_by_class('test', y_test)

100%|██████████| 1473/1473 [01:17<00:00, 18.92it/s]


In [ ]:
save_images_by_class('train', y_train)